In [0]:
#checking for mounted points
display(dbutils.fs.mounts())

In [0]:
#fetching mounted point salespoint as named for mount point
dbutils.fs.ls("/mnt/Salespoint")

In [0]:
dbutils.fs.ls("/mnt/Salespoint/Salestotalrevenuedelta")

In [0]:
#how to create delta table?
Salesdelta = spark.table("delta.`/mnt/Salespoint/Salestotalrevenuedelta'")
display(Salesdelta)


In [0]:
#reading delta table from mount point
df = spark.read.format("delta").load("/mnt/Salespoint/Salestotalrevenuedelta")
display(df)

In [0]:
%sql
--creating delta table using sql
CREATE TABLE Salesdelta USING DELTA LOCATION '/mnt/Salespoint/Salestotalrevenuedelta'

In [0]:
%sql
select * from Salesdelta


In [0]:
%sql
--describing the table
DESCRIBE DETAIL Salesdelta

In [0]:
%sql
--to check the schema
DESCRIBE FORMATTED Salesdelta

In [0]:
%python
# Writing the main df dataframe to delta table partitioned by region
df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("Region") \
    .save("/mnt/Salespoint/Salestotalrevenuedelta")

In [0]:
%sql
--fetech the deatils of the table
SHOW TABLES

In [0]:
spark.sql("DESCRIBE FORMATTED Salesdelta")

In [0]:
%sql
-- we can see all the partioned data in same file again with partions
select * from Salesdelta

In [0]:
%sql
--describe Salesdelta
DESCRIBE Salesdelta

In [0]:
%sql
--describe history of Salesdelta
describe history Salesdelta

In [0]:
%sql
--showing version of Salesdelta
select * from Salesdelta VERSION AS OF 1

In [0]:
%sql
-- delting some regions from table like ASIA
delete from Salesdelta where Region = 'Asia'

In [0]:
%sql
select * from Salesdelta

In [0]:
%sql
--describing history of sales date afters deleting region asia from the table
describe history Salesdelta

In [0]:
%sql
-- writing back the deleted region Asia into main table 
INSERT INTO Salesdelta 
SELECT * 
FROM Salesdelta VERSION AS OF 1 WHERE Region = 'Asia'

In [0]:
%sql
-- Now seeing the main table after retrving the deleted region asia
select * from Salesdelta

In [0]:
%sql
--history of salesdelta after retriving the deleted region asia
describe history Salesdelta

In [0]:
#reading delta table using versions and time stamp
df = spark.read.format("delta").option("versionAsOf", 3).load("/mnt/Salespoint/Salestotalrevenuedelta")
display(df)

In [0]:
#creating random data to upsert the current Salesdelta table
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
import random

# Define the schema for the DataFrame
schema = ["Region", "ItemType", "SalesChannel", "TotalRevenue"]

# Define possible values for each column
regions = ["North America", "Europe", "Asia", "South America", "Africa"]
item_types = ["Electronics", "Clothing", "Food", "Furniture", "Toys"]
sales_channels = ["Online", "Offline"]
total_profits = [round(random.uniform(1000, 10000), 2) for _ in range(100)]

# Generate random data
data = [(random.choice(regions), random.choice(item_types), random.choice(sales_channels), random.choice(total_profits)) for _ in range(100)]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.show()

In [0]:
display(df)

In [0]:
%python
# Create a temporary view from the DataFrame `df`
df.createOrReplaceTempView("df")

In [0]:
%sql
--merging df data in Salesdelta (SCD Type 1)
MERGE INTO Salesdelta
USING df
ON Salesdelta.Region = df.Region AND Salesdelta.ItemType = df.ItemType AND Salesdelta.SalesChannel = df.SalesChannel
WHEN MATCHED THEN UPDATE SET TotalRevenue = df.TotalRevenue 
WHEN NOT MATCHED THEN INSERT (Region, ItemType, SalesChannel, TotalRevenue) VALUES (df.Region, df.ItemType, df.SalesChannel, df.TotalRevenue)

In [0]:
%sql
--describing history of salesdelta after merging df data in Salesdelta (SCD Type 1)
describe history Salesdelta